# Tabular data training and serving with Keras and Ray AIR

This notebook is adapted from [a Keras tutorial](https://www.tensorflow.org/tfx/tutorials/tfx/components_keras).
It uses [Chicago Taxi dataset](https://data.cityofchicago.org/Transportation/Taxi-Trips/wrvz-psew) and a DNN Keras model to predict whether a trip may generate a big tip.

In this example, we showcase how to achieve the same tasks as the Keras Tutorial using [Ray AIR](https://docs.ray.io/en/latest/ray-air/getting-started.html), covering
every step from data ingestion to pushing a model to serving.

1. Read a CSV into [Ray Dataset](https://docs.ray.io/en/latest/data/dataset.html).
2. Process the dataset by chaining [Ray AIR preprocessors](https://docs.ray.io/en/latest/ray-air/getting-started.html#preprocessors).
3. Train the model using the TensorflowTrainer from AIR.
4. Serve the model using Ray Serve and the above preprocessors.

Uncomment and run the following line in order to install all the necessary dependencies:

In [1]:
# ! pip install "tensorflow>=2.8.0" "ray[tune, data, serve]>=1.12.1"
# ! pip install fastapi

## Set up Ray

We will use `ray.init()` to initialize a local cluster. By default, this cluster will be composed of only the machine you are running this notebook on. If you wish to attach to an existing Ray cluster, you can do so through `ray.init(address="auto")`.

In [2]:
from pprint import pprint
import ray

ray.shutdown()
ray.init()

RayContext(dashboard_url='127.0.0.1:8265', python_version='3.7.7', ray_version='3.0.0.dev0', ray_commit='67cd984b92b50a028fe2b672cf713c4205361354', address_info={'node_ip_address': '172.31.70.13', 'raylet_ip_address': '172.31.70.13', 'redis_address': None, 'object_store_address': '/tmp/ray/session_2022-05-25_13-39-26_703288_182/sockets/plasma_store', 'raylet_socket_name': '/tmp/ray/session_2022-05-25_13-39-26_703288_182/sockets/raylet', 'webui_url': '127.0.0.1:8265', 'session_dir': '/tmp/ray/session_2022-05-25_13-39-26_703288_182', 'metrics_export_port': 57149, 'gcs_address': '172.31.70.13:9031', 'address': '172.31.70.13:9031', 'node_id': '18f82d1c3eaf9dccb9a1813246ace59f0505ea9128ecc969f4aa7ffa'})

We can check the resources our cluster is composed of. If you are running this notebook on your local machine or Google Colab, you should see the number of CPU cores and GPUs available on the said machine.

In [3]:
pprint(ray.cluster_resources())

{'CPU': 8.0,
 'memory': 19643840103.0,
 'node:172.31.70.13': 1.0,
 'object_store_memory': 9821920051.0}


## Getting the data

Let's start with defining a helper function to get the data to work with. Some columns are dropped for simplicity.

In [4]:
import pandas as pd

LABEL = "is_big_tip"

def get_data() -> pd.DataFrame:
    """Fetch the taxi fare data to work on."""
    _data = pd.read_csv(
        "https://raw.githubusercontent.com/tensorflow/tfx/master/"
        "tfx/examples/chicago_taxi_pipeline/data/simple/data.csv"
    )
    _data[LABEL] = _data["tips"] / _data["fare"] > 0.2
    # We drop some columns here for the sake of simplicity.
    return _data.drop(
        [
            "tips",
            "fare",
            "dropoff_latitude",
            "dropoff_longitude",
            "pickup_latitude",
            "pickup_longitude",
            "pickup_census_tract",
        ],
        axis=1,
    )

In [5]:
data = get_data()

Now let's take a look at the data. Notice that some values are missing. This is exactly where preprocessing comes into the picture. We will come back to this in the preprocessing session below.

In [6]:
data.head(5)

,pickup_community_area,trip_start_month,trip_start_hour,trip_start_day,trip_start_timestamp,trip_miles,dropoff_census_tract,payment_type,company,trip_seconds,dropoff_community_area,is_big_tip
0,NaN,5,19,6,1400269500,0.0,NaN,Credit Card,Chicago Elite Cab Corp. (Chicago Carriag,0.0,NaN,False
1,NaN,3,19,5,1362683700,0.0,NaN,Unknown,Chicago Elite Cab Corp.,300.0,NaN,False
2,60.0,10,2,3,1380593700,12.6,NaN,Cash,Taxi Affiliation Services,1380.0,NaN,False
3,10.0,10,1,2,1382319000,0.0,NaN,Cash,Taxi Affiliation Services,180.0,NaN,False
4,14.0,5,7,5,1369897200,0.0,NaN,Cash,Dispatch Taxi Affiliation,1080.0,NaN,False


We continue to split the data into training and test data.
For the test data, we separate out the features to run serving on as well as labels to compare serving results with.

In [7]:
import numpy as np
from sklearn.model_selection import train_test_split
from typing import Tuple


def split_data(data: pd.DataFrame) -> Tuple[ray.data.Dataset, pd.DataFrame, np.array]:
    """Split the data in a stratified way.

    Returns:
        A tuple containing train dataset, test data and test label.
    """
    # There is a native offering in Ray Dataset for split as well.
    # However, supporting stratification is a TODO there. So use
    # scikit-learn equivalent here.
    train_data, test_data = train_test_split(
        data, stratify=data[[LABEL]], random_state=1113
    )
    _train_ds = ray.data.from_pandas(train_data)
    _test_label = test_data[LABEL].values
    _test_df = test_data.drop([LABEL], axis=1)
    return _train_ds, _test_df, _test_label

train_ds, test_df, test_label = split_data(data)

In [8]:
print(f"There are {train_ds.count()} samples for training and {test_df.shape[0]} samples for testing.")

There are 11251 samples for training and 3751 samples for testing.


## Preprocessing

Let's focus on preprocessing first.
Usually, input data needs to go through some preprocessing before being
fed into model. It is a good idea to package preprocessing logic into
a modularized component so that the same logic can be applied to both
training data as well as data for online serving or offline batch prediction.

In AIR, this component is a [`Preprocessor`](https://docs.ray.io/en/latest/ray-air/getting-started.html#preprocessors).
It is constructed in a way that allows easy composition.

Now let's construct a chained preprocessor composed of simple preprocessors, including
1. Imputer for filling missing features;
2. OneHotEncoder for encoding categorical features;
3. BatchMapper where arbitrary user-defined function can be applied to batches of records;
and so on. Take a look at [`Preprocessor`](https://docs.ray.io/en/latest/ray-air/getting-started.html#preprocessors).
The output of the preprocessing step goes into model for training.

In [9]:
from ray.data.preprocessors import (
    BatchMapper,
    Chain,
    OneHotEncoder,
    SimpleImputer,
)

def get_preprocessor():
    """Construct a chain of preprocessors."""
    imputer1 = SimpleImputer(
        ["dropoff_census_tract"], strategy="most_frequent"
    )
    imputer2 = SimpleImputer(
        ["pickup_community_area", "dropoff_community_area"],
        strategy="most_frequent",
    )
    imputer3 = SimpleImputer(["payment_type"], strategy="most_frequent")
    imputer4 = SimpleImputer(
        ["company"], strategy="most_frequent")
    imputer5 = SimpleImputer(
        ["trip_start_timestamp", "trip_miles", "trip_seconds"], strategy="mean"
    )

    ohe = OneHotEncoder(
        columns=[
            "trip_start_hour",
            "trip_start_day",
            "trip_start_month",
            "dropoff_census_tract",
            "pickup_community_area",
            "dropoff_community_area",
            "payment_type",
            "company",
        ],
        limit={
            "dropoff_census_tract": 25,
            "pickup_community_area": 20,
            "dropoff_community_area": 20,
            "payment_type": 2,
            "company": 7,
        },
    )

    def batch_mapper_fn(df):
        df["trip_start_year"] = pd.to_datetime(df["trip_start_timestamp"], unit="s").dt.year
        df = df.drop(["trip_start_timestamp"], axis=1)
        return df

    chained_pp = Chain(
        imputer1,
        imputer2,
        imputer3,
        imputer4,
        imputer5,
        ohe,
        BatchMapper(batch_mapper_fn),
    )
    return chained_pp


Now let's define some constants for clarity.

In [10]:
# Note that `INPUT_SIZE` here is corresponding to the output dimension
# of the previously defined processing steps.
# This is used to specify the input shape of Keras model as well as
# when converting from training data from `ray.data.Dataset` to `tf.Tensor`.
INPUT_SIZE = 120
# The training batch size. Based on `NUM_WORKERS`, each worker
# will get its own share of this batch size. For example, if
# `NUM_WORKERS = 2`, each worker will work on 4 samples per batch.
BATCH_SIZE = 8
# Number of epoch. Adjust it based on how quickly you want the run to be.
EPOCH = 1
# Number of training workers.
# Adjust this accordingly based on the resources you have!
NUM_WORKERS = 2

## Training

Let's starting with defining a simple Keras model for the classification task.

In [11]:
import tensorflow as tf

def build_model():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.Input(shape=(INPUT_SIZE,)))
    model.add(tf.keras.layers.Dense(50, activation="relu"))
    model.add(tf.keras.layers.Dense(1, activation="sigmoid"))
    return model

Now let's define the training loop. This code will be run on each training
worker in a distributed fashion. See more details [here](https://docs.ray.io/en/latest/train/train.html).

In [12]:
from ray import train
from ray.train.tensorflow import prepare_dataset_shard
from ray.tune.integration.keras import TuneReportCallback

def train_loop_per_worker():
    dataset_shard = train.get_dataset_shard("train")

    strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()
    with strategy.scope():
        model = build_model()
        model.compile(
            loss="binary_crossentropy",
            optimizer="adam",
            metrics=["accuracy"],
        )

    for epoch in range(EPOCH):
        # This will make sure that the training workers will get their own
        # share of batch to work on.
        # See `ray.train.tensorflow.prepare_dataset_shard` for more information.
        tf_dataset = prepare_dataset_shard(
            dataset_shard.to_tf(
                label_column=LABEL,
                output_signature=(
                    tf.TensorSpec(shape=(BATCH_SIZE, INPUT_SIZE), dtype=tf.float32),
                    tf.TensorSpec(shape=(BATCH_SIZE,), dtype=tf.int64),
                ),
                batch_size=BATCH_SIZE,
                drop_last=True,
            )
        )

        model.fit(tf_dataset, verbose=0)
        # This saves checkpoint in a way that can be used by Ray Serve coherently.
        train.save_checkpoint(epoch=epoch, model=model.get_weights())

Now let's define a trainer that takes in the training loop,
the training dataset as well the preprocessor that we just defined.

And run it!

Notice that you can tune how long you want the run to be by changing ``EPOCH``.

In [13]:
from ray.train.tensorflow import TensorflowTrainer

trainer = TensorflowTrainer(
    train_loop_per_worker=train_loop_per_worker,
    scaling_config={"num_workers": NUM_WORKERS},
    datasets={"train": train_ds},
    preprocessor=get_preprocessor(),
)
result = trainer.fit()

Trial name,status,loc
TensorflowTrainer_05521_00000,TERMINATED,172.31.70.13:961


(BaseWorkerMixin pid=1066) WARNING:tensorflow:From /tmp/ipykernel_823/2334437326.py:8: _CollectiveAllReduceStrategyExperimental.__init__ (from tensorflow.python.distribute.collective_all_reduce_strategy) is deprecated and will be removed in a future version.
(BaseWorkerMixin pid=1066) Instructions for updating:
(BaseWorkerMixin pid=1066) use distribute.MultiWorkerMirroredStrategy instead
(BaseWorkerMixin pid=1066) 2022-05-25 13:41:25.540330: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
(BaseWorkerMixin pid=1066) 2022-05-25 13:41:25.540362: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
(BaseWorkerMixin pid=1066) 2022-05-25 13:41:25.540389: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to

Trial TensorflowTrainer_05521_00000 completed. Last result: 


2022-05-25 13:41:39,516	INFO tune.py:753 -- Total run time: 24.49 seconds (23.53 seconds for the tuning loop).


## Moving on to Serve

We will use Ray Serve to serve the trained model. A core concept of Ray Serve is [Deployment](https://docs.ray.io/en/latest/serve/core-apis.html). It allows you to define and update your business logic or models that will handle incoming requests as well as how this is exposed over HTTP or in Python.

In the case of serving model, `ray.serve.model_wrappers.ModelWrapper` and `ray.serve.model_wrappers.ModelWrapperDeployment` wrap a `ray.air.checkpoint.Checkpoint` into a Ray Serve deployment that can readily serve HTTP requests.
Note, ``Checkpoint`` captures both model and preprocessing steps in a way compatible with Ray Serve and ensures that ml workload can transition seamlessly between training and
serving.

This removes the boilerplate code and minimizes the effort to bring your model to production!

Generally speaking the http request can either send in json or data.
Upon receiving this payload, Ray Serve would need some "adapter" to convert the request payload into some shape or form that can be accepted as input to the preprocessing steps. In this case, we send in a json request and converts it to a pandas DataFrame through `dataframe_adapter`, defined below.

In [14]:
from fastapi import Request

async def dataframe_adapter(request: Request):
    """Serve HTTP Adapter that reads JSON and converts to pandas DataFrame."""
    content = await request.json()
    return pd.DataFrame.from_dict(content)

Now let's wrap everything in a serve endpoint that exposes a URL to where requests can be sent to.

In [15]:
from ray import serve
from ray.air.checkpoint import Checkpoint
from ray.train.tensorflow import TensorflowPredictor
from ray.serve.model_wrappers import ModelWrapperDeployment


def serve_model(checkpoint: Checkpoint, model_definition, adapter, name="Model") -> str:
    """Expose a serve endpoint.

    Returns:
        serve URL.
    """
    serve.start(detached=True)
    deployment = ModelWrapperDeployment.options(name=name)
    deployment.deploy(
        TensorflowPredictor,
        checkpoint,
        # This is due to a current limitation on Serve that's
        # being addressed.
        # TODO(xwjiang): Change to True.
        batching_params=False,
        model_definition=model_definition,
        http_adapter=adapter,
    )
    return deployment.url


In [16]:
# Generally speaking, training and serving are done in totally different ray clusters.
# To simulate that, let's shutdown the old ray cluster in preparation for serving.
ray.shutdown()

endpoint_uri = serve_model(result.checkpoint, build_model, dataframe_adapter)

2022-05-25 13:41:39,979	INFO worker.py:863 -- Using address localhost:9031 set in the environment variable RAY_ADDRESS
2022-05-25 13:41:39,981	INFO worker.py:965 -- Connecting to existing Ray cluster at address: 172.31.70.13:9031
(ServeController pid=1582) INFO 2022-05-25 13:41:41,026 controller 1582 checkpoint_path.py:17 - Using RayInternalKVStore for controller checkpoint and recovery.
(ServeController pid=1582) INFO 2022-05-25 13:41:41,028 controller 1582 http_state.py:115 - Starting HTTP proxy with name 'SERVE_CONTROLLER_ACTOR:SERVE_PROXY_ACTOR-node:172.31.70.13-0' on node 'node:172.31.70.13-0' listening on '127.0.0.1:8000'
(HTTPProxyActor pid=1616) INFO:     Started server process [1616]
(ServeController pid=1582) INFO 2022-05-25 13:41:43,907 controller 1582 deployment_state.py:1219 - Adding 1 replicas to deployment 'Model'.


Let's write a helper function to send requests to this endpoint and compare the results with labels.

In [17]:
import requests

NUM_SERVE_REQUESTS = 100

def send_requests(df: pd.DataFrame, label: np.array):
    for i in range(NUM_SERVE_REQUESTS):
        one_row = df.iloc[[i]].to_dict()
        serve_result = requests.post(endpoint_uri, json=one_row).json()
        print(
            f"request[{i}] prediction: {serve_result['predictions']['0']} "
            f"- label: {str(label[i])}"
        )

In [18]:
send_requests(test_df, test_label)

(HTTPProxyActor pid=1616) INFO 2022-05-25 13:41:46,962 http_proxy 172.31.70.13 http_proxy.py:320 - POST /Model 307 4.3ms
(Model pid=1661) INFO 2022-05-25 13:41:46,961 Model Model#ryhdIZ replica.py:483 - HANDLE __call__ OK 0.3ms
(Model pid=1661) 2022-05-25 13:41:47.030073: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
(Model pid=1661) 2022-05-25 13:41:47.030109: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
(Model pid=1661) 2022-05-25 13:41:47.030127: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ip-172-31-70-13): /proc/driver/nvidia/version does not exist
(Model pid=1661) 2022-05-25 13:41:47.030420: I tensorflow/core/platform/cpu_feature_guard.cc:142] This Ten

request[0] prediction: [0.06145152449607849] - label: True
request[1] prediction: [0.035036444664001465] - label: False
request[2] prediction: [0.00014954805374145508] - label: False


(HTTPProxyActor pid=1616) INFO 2022-05-25 13:41:47,152 http_proxy 172.31.70.13 http_proxy.py:320 - POST /Model 200 81.5ms
(HTTPProxyActor pid=1616) INFO 2022-05-25 13:41:47,159 http_proxy 172.31.70.13 http_proxy.py:320 - POST /Model 307 2.6ms
(HTTPProxyActor pid=1616) INFO 2022-05-25 13:41:47,244 http_proxy 172.31.70.13 http_proxy.py:320 - POST /Model 200 82.4ms
(Model pid=1661) INFO 2022-05-25 13:41:47,151 Model Model#ryhdIZ replica.py:483 - HANDLE __call__ OK 79.0ms
(Model pid=1661) INFO 2022-05-25 13:41:47,158 Model Model#ryhdIZ replica.py:483 - HANDLE __call__ OK 0.3ms
(Model pid=1661) INFO 2022-05-25 13:41:47,242 Model Model#ryhdIZ replica.py:483 - HANDLE __call__ OK 79.9ms
(HTTPProxyActor pid=1616) INFO 2022-05-25 13:41:47,251 http_proxy 172.31.70.13 http_proxy.py:320 - POST /Model 307 2.7ms
(HTTPProxyActor pid=1616) INFO 2022-05-25 13:41:47,340 http_proxy 172.31.70.13 http_proxy.py:320 - POST /Model 200 86.7ms
(Model pid=1661) INFO 2022-05-25 13:41:47,250 Model Model#ryhdIZ repl

request[3] prediction: [0.00040149688720703125] - label: False
request[4] prediction: [0.0006460845470428467] - label: False
request[5] prediction: [5.145270915818401e-05] - label: False


(HTTPProxyActor pid=1616) INFO 2022-05-25 13:41:47,348 http_proxy 172.31.70.13 http_proxy.py:320 - POST /Model 307 3.1ms
(HTTPProxyActor pid=1616) INFO 2022-05-25 13:41:47,432 http_proxy 172.31.70.13 http_proxy.py:320 - POST /Model 200 82.0ms
(HTTPProxyActor pid=1616) INFO 2022-05-25 13:41:47,439 http_proxy 172.31.70.13 http_proxy.py:320 - POST /Model 307 2.7ms
(Model pid=1661) INFO 2022-05-25 13:41:47,431 Model Model#ryhdIZ replica.py:483 - HANDLE __call__ OK 79.5ms
(Model pid=1661) INFO 2022-05-25 13:41:47,438 Model Model#ryhdIZ replica.py:483 - HANDLE __call__ OK 0.3ms
(HTTPProxyActor pid=1616) INFO 2022-05-25 13:41:47,524 http_proxy 172.31.70.13 http_proxy.py:320 - POST /Model 200 82.6ms
(HTTPProxyActor pid=1616) INFO 2022-05-25 13:41:47,531 http_proxy 172.31.70.13 http_proxy.py:320 - POST /Model 307 2.9ms
(Model pid=1661) INFO 2022-05-25 13:41:47,522 Model Model#ryhdIZ replica.py:483 - HANDLE __call__ OK 80.1ms
(Model pid=1661) INFO 2022-05-25 13:41:47,529 Model Model#ryhdIZ repli

...

(HTTPProxyActor pid=1616) INFO 2022-05-25 13:41:55,652 http_proxy 172.31.70.13 http_proxy.py:320 - POST /Model 200 83.4ms
(HTTPProxyActor pid=1616) INFO 2022-05-25 13:41:55,659 http_proxy 172.31.70.13 http_proxy.py:320 - POST /Model 307 2.5ms
(Model pid=1661) INFO 2022-05-25 13:41:55,651 Model Model#ryhdIZ replica.py:483 - HANDLE __call__ OK 80.8ms
(Model pid=1661) INFO 2022-05-25 13:41:55,658 Model Model#ryhdIZ replica.py:483 - HANDLE __call__ OK 0.3ms
(HTTPProxyActor pid=1616) INFO 2022-05-25 13:41:55,744 http_proxy 172.31.70.13 http_proxy.py:320 - POST /Model 200 82.7ms
(HTTPProxyActor pid=1616) INFO 2022-05-25 13:41:55,750 http_proxy 172.31.70.13 http_proxy.py:320 - POST /Model 307 2.7ms
(Model pid=1661) INFO 2022-05-25 13:41:55,742 Model Model#ryhdIZ replica.py:483 - HANDLE __call__ OK 80.1ms
(Model pid=1661) INFO 2022-05-25 13:41:55,749 Model Model#ryhdIZ replica.py:483 - HANDLE __call__ OK 0.3ms


request[93] prediction: [0.0001958608627319336] - label: False
request[94] prediction: [0.00015294551849365234] - label: False
request[95] prediction: [0.15767309069633484] - label: True


(HTTPProxyActor pid=1616) INFO 2022-05-25 13:41:55,839 http_proxy 172.31.70.13 http_proxy.py:320 - POST /Model 200 86.2ms
(HTTPProxyActor pid=1616) INFO 2022-05-25 13:41:55,846 http_proxy 172.31.70.13 http_proxy.py:320 - POST /Model 307 2.6ms
(Model pid=1661) INFO 2022-05-25 13:41:55,838 Model Model#ryhdIZ replica.py:483 - HANDLE __call__ OK 83.5ms
(Model pid=1661) INFO 2022-05-25 13:41:55,845 Model Model#ryhdIZ replica.py:483 - HANDLE __call__ OK 0.3ms
(HTTPProxyActor pid=1616) INFO 2022-05-25 13:41:55,935 http_proxy 172.31.70.13 http_proxy.py:320 - POST /Model 200 86.1ms
(HTTPProxyActor pid=1616) INFO 2022-05-25 13:41:55,942 http_proxy 172.31.70.13 http_proxy.py:320 - POST /Model 307 2.8ms
(Model pid=1661) INFO 2022-05-25 13:41:55,933 Model Model#ryhdIZ replica.py:483 - HANDLE __call__ OK 83.5ms
(Model pid=1661) INFO 2022-05-25 13:41:55,940 Model Model#ryhdIZ replica.py:483 - HANDLE __call__ OK 0.3ms
(HTTPProxyActor pid=1616) INFO 2022-05-25 13:41:56,028 http_proxy 172.31.70.13 http_

request[96] prediction: [0.17295515537261963] - label: True
request[97] prediction: [0.00022396445274353027] - label: False
request[98] prediction: [0.0002429187297821045] - label: False
request[99] prediction: [0.7572120428085327] - label: True


(HTTPProxyActor pid=1616) INFO 2022-05-25 13:41:56,221 http_proxy 172.31.70.13 http_proxy.py:320 - POST /Model 200 87.6ms
(HTTPProxyActor pid=1616) INFO 2022-05-25 13:41:56,228 http_proxy 172.31.70.13 http_proxy.py:320 - POST /Model 307 2.9ms
(Model pid=1661) INFO 2022-05-25 13:41:56,220 Model Model#ryhdIZ replica.py:483 - HANDLE __call__ OK 84.7ms
(Model pid=1661) INFO 2022-05-25 13:41:56,227 Model Model#ryhdIZ replica.py:483 - HANDLE __call__ OK 0.3ms
(HTTPProxyActor pid=1616) INFO 2022-05-25 13:41:56,316 http_proxy 172.31.70.13 http_proxy.py:320 - POST /Model 200 85.3ms
(HTTPProxyActor pid=1616) INFO 2022-05-25 13:41:56,323 http_proxy 172.31.70.13 http_proxy.py:320 - POST /Model 307 2.9ms
(Model pid=1661) INFO 2022-05-25 13:41:56,315 Model Model#ryhdIZ replica.py:483 - HANDLE __call__ OK 82.4ms
(Model pid=1661) INFO 2022-05-25 13:41:56,322 Model Model#ryhdIZ replica.py:483 - HANDLE __call__ OK 0.3ms
(HTTPProxyActor pid=1616) INFO 2022-05-25 13:41:56,408 http_proxy 172.31.70.13 http_